<a href="https://colab.research.google.com/github/DhruvK278/Invoice-OCR/blob/main/OCR_invoice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install necessary libraries and define your API key

import requests
import pandas as pd

YOUR_API_KEY = 'K86554537188957'

print("✅ Setup Complete. API key is set.")

✅ Setup Complete. API key is set.


In [2]:
# Cell 2: Upload the invoice file to Colab

from google.colab import files

print("Please upload your invoice image file:")
uploaded = files.upload()

if uploaded:
    file_path = next(iter(uploaded))
    print(f"\n✅ Successfully uploaded '{file_path}'")
else:
    print("\n⚠️ No file uploaded.")
    file_path = None

Please upload your invoice image file:


Saving invoice 2.jpg to invoice 2.jpg

✅ Successfully uploaded 'invoice 2.jpg'


In [3]:
# Cell 3: Send the OCR request to the API (Corrected)

if file_path and YOUR_API_KEY != 'helloworld':
    try:
        with open(file_path, 'rb') as f:
            # Define the API endpoint and payload
            url_api = 'https://api.ocr.space/parse/image'
            payload = {
                'apikey': YOUR_API_KEY,
                'language': 'eng',
                'isOverlayRequired': True,
                'detectOrientation': True,
                'scale': True,
                'isTable': True,
            }

            # Send the POST request
            result = requests.post(url_api,
                                   files={file_path: f},
                                   data=payload,
                                   )

        # Decode the JSON response
        ocr_result = result.json()

        # Check if the OCR was successful
        if ocr_result.get('IsErroredOnProcessing'):
            print("--- ⚠️ Error in OCR Processing ---")
            print(ocr_result.get('ErrorMessage'))
            parsed_text = None
        else:
            print("✅ OCR processing successful!")
            # Extract the parsed text from the first result
            parsed_text = ocr_result['ParsedResults'][0]['ParsedText']

    except Exception as e:
        print(f"An error occurred: {e}")
        parsed_text = None
else:
    print("\n⚠️ Please upload a file and set your API key in Cell 1 before running this cell.")
    parsed_text = None

✅ OCR processing successful!


In [4]:
# Cell 4: Display the raw text returned by the API

if 'parsed_text' in locals() and parsed_text:
    print("\n--- RAW TEXT FROM API ---")
    print(parsed_text)
    print("-------------------------")


--- RAW TEXT FROM API ---
Invoice no.	• 59326638	
Date of issue:	05/27/2013	
Seller:	Client:	
Ashley-Shaw	Huber, Walker and Mullins	
USNS Wilson	1892 Jeffrey Avenue Suite 966	
FPO AE 36740	Heatherberg, NC 86915	
Tax ld: 926-83-7399	Tax ld: 929-76-2624	
IBAN: GB39MTZP97412080764188	
ITEMS	
No.	Description	Qty	Net price	Net worth	VAT [0/0]	Gross	
worth	
1.	Obama An Intimate Portrait by	5,00	each	4,99	24,95	27,45	
Pete Souza P-F EßOOK	
2.	My Passion for Design by Barbra	2,00	each	5,56	11,12	12,23	
Streisand	
3.	Entertaining	4,00	each	4,49	17,96	19,76	
4.	Most famous bracelet in the	1,00	each	20,00	20,00	22,00	
world	
5.	Straw Dogs : Thoughts on	5,00	each	13,99	69,95	76,95	
Humans and Other Animals,	
Paperback by Gray, John, L...	
SUMMARY	
VAT [0/0]	Net worth	VAT	Gross worth	
10%	143,98	14,40	158,38	
Total	$ 143,98	$ 14,40	$ 158,38	

-------------------------


In [5]:
# Cell 5 (Final Corrected Logic): Parse the Structured Raw Text
import re

if 'parsed_text' in locals() and parsed_text:
    print("\n--- Parsing with final logic to handle multi-line descriptions ---")

    lines = parsed_text.strip().split('\n')
    line_items = []

    # --- Logic to find and parse the items table ---
    in_items_section = False

    for line in lines:
        clean_line = line.strip()
        if not clean_line:
            continue

        # Start capturing when we see the table headers
        if "No." in clean_line and "Description" in clean_line:
            in_items_section = True
            # Define the headers based on this line
            headers = [h.strip() for h in clean_line.split('\t')]
            continue

        # Stop when we reach the summary
        if "SUMMARY" in clean_line:
            in_items_section = False
            break

        if in_items_section:
            # THIS IS THE CORRECTED LOGIC:
            # A new item must start with a digit followed by a dot or comma.
            if re.match(r'^\d[\.,]', clean_line):
                # This is a new line item. Parse it.
                values = [v.strip() for v in clean_line.split('\t')]
                # Pad values in case some columns are empty for this row
                padded_values = values + [None] * (len(headers) - len(values))
                row_data = dict(zip(headers, padded_values))
                line_items.append(row_data)

            # If the line does NOT start with the new item pattern, it's a continuation.
            elif line_items:
                # Append this text to the 'Description' of the *last* item we added.
                line_items[-1]['Description'] += " " + clean_line

    # --- Display the final, correct DataFrame ---
    if line_items:
        print("\n--- DETECTED LINE ITEMS (Corrected) ---")
        df = pd.DataFrame(line_items)

        # Select and reorder columns for a cleaner look
        display_columns = ['No.', 'Description', 'Qty', 'Net price', 'Net worth']
        # Filter to only show columns that actually exist in the DataFrame
        final_columns = [col for col in display_columns if col in df.columns]

        display(df[final_columns])
    else:
        print("\n--- ⚠️ No structured line items were detected from the raw text. ---")


--- Parsing with final logic to handle multi-line descriptions ---

--- DETECTED LINE ITEMS (Corrected) ---


,No.,Description,Qty,Net price,Net worth
0,1.,Obama An Intimate Portrait by Pete Souza P-F E...,"5,00",each,"4,99"
1,2.,My Passion for Design by Barbra Streisand,"2,00",each,"5,56"
2,3.,Entertaining,"4,00",each,"4,49"
3,4.,Most famous bracelet in the world,"1,00",each,"20,00"
4,5.,Straw Dogs : Thoughts on Humans and Other Anim...,"5,00",each,"13,99"


In [6]:
# Cell to install the PostgreSQL driver
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.7 MB/s eta 0:00:00


In [10]:
# FINAL CELL: Parse Data and Insert into PostgreSQL

import psycopg2
import re
import pandas as pd

# --- Make sure the OCR result from the API exists before we do anything ---
if 'ocr_result' not in locals() or ocr_result.get('IsErroredOnProcessing'):
    print("⚠️ OCR processing failed or was not run. Cannot proceed to database insertion.")
else:
    # ===================================================================
    # PART 1: PARSE THE OCR TEXT INTO A STRUCTURED FORMAT
    # ===================================================================
    print("--- Step 1: Parsing the OCR text ---")

    parsed_text = ocr_result['ParsedResults'][0]['ParsedText']
    lines = parsed_text.strip().split('\n')
    line_items_data = []
    parsed_summary_data = {}

    in_items_section = False
    headers = []

    for line in lines:
        clean_line = line.strip()
        if not clean_line:
            continue

        # --- Find summary data (Invoice Number, Date) ---
        if "Invoice no:" in clean_line:
            parsed_summary_data['InvoiceNumber'] = clean_line.split(":")[-1].strip()
        if "Date of issue:" in clean_line:
            parsed_summary_data['InvoiceDate'] = clean_line.split(":")[-1].strip()

        # --- Parse the table of line items ---
        if "No." in clean_line and "Description" in clean_line:
            in_items_section = True
            headers = [h.strip() for h in clean_line.split('\t')]
            continue

        if "SUMMARY" in clean_line:
            in_items_section = False
            break

        if in_items_section:
            if re.match(r'^\d[\.,]', clean_line):
                values = [v.strip() for v in clean_line.split('\t')]
                padded_values = values + [None] * (len(headers) - len(values))
                row_data = dict(zip(headers, padded_values))
                line_items_data.append(row_data)
            elif line_items_data:
                line_items_data[-1]['Description'] += " " + clean_line

    # Create the final DataFrame
    df = pd.DataFrame(line_items_data)
    print("✅ Parsing complete. Displaying final line items:")
    display(df)

    # ===================================================================
    # PART 2: CONNECT TO POSTGRESQL AND INSERT THE DATA
    # ===================================================================
    print("\n--- Step 2: Inserting data into PostgreSQL ---")

    # --- Fill in your database credentials ---
    db_params = {
        "host": "localhost",
        "dbname": "invoice_data",
        "user": "postgres",
        "password": "dk123" # <-- IMPORTANT: CHANGE THIS
    }

    conn = None
    try:
        print("Connecting to the database...")
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()

        # --- Insert into the main 'invoices' table ---
        sql_invoice = "INSERT INTO invoices (invoice_number, invoice_date) VALUES (%s, %s) RETURNING id;"
        cur.execute(sql_invoice, (
            parsed_summary_data.get('InvoiceNumber'),
            parsed_summary_data.get('InvoiceDate'),
        ))
        invoice_id = cur.fetchone()[0]
        print(f"✅ Main invoice details saved. New Invoice ID: {invoice_id}")

        # --- Insert all line items from the DataFrame ---
        print(f"Saving {len(df)} line items...")
        for index, row in df.iterrows():
            sql_line_item = """
                INSERT INTO line_items (invoice_id, item_number, description, quantity, net_price, net_worth)
                VALUES (%s, %s, %s, %s, %s, %s);
            """
            cur.execute(sql_line_item, (
                invoice_id, row.get('No.'), row.get('Description'), row.get('Qty'),
                row.get('Net price'), row.get('Net worth')
            ))

        conn.commit()
        print("✅ Transaction successful! All data has been saved to the database.")

    except Exception as e:
        print(f"--- ⚠️ DATABASE ERROR ---")
        print(f"An error occurred: {e}")
        if conn:
            conn.rollback() # Roll back changes if something went wrong

    finally:
        if conn:
            cur.close()
            conn.close()
            print("Database connection closed.")

--- Step 1: Parsing the OCR text ---
✅ Parsing complete. Displaying final line items:


,No.,Description,Qty,Net price,Net worth,VAT [0/0],Gross
0,1.,Obama An Intimate Portrait by Pete Souza P-F E...,"5,00",each,"4,99","24,95","27,45"
1,2.,My Passion for Design by Barbra Streisand,"2,00",each,"5,56","11,12","12,23"
2,3.,Entertaining,"4,00",each,"4,49","17,96","19,76"
3,4.,Most famous bracelet in the world,"1,00",each,"20,00","20,00","22,00"
4,5.,Straw Dogs : Thoughts on Humans and Other Anim...,"5,00",each,"13,99","69,95","76,95"



--- Step 2: Inserting data into PostgreSQL ---
Connecting to the database...
--- ⚠️ DATABASE ERROR ---
An error occurred: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (::1), port 5432 failed: Cannot assign requested address
	Is the server running on that host and accepting TCP/IP connections?

